<a href="https://colab.research.google.com/github/ml2-picme/PicMe/blob/master/Email_Download_and_Examining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import load_img, img_to_array
import urllib.request

def downloadEmailFromUrl(URL, localPath):
  print("Lade ", URL, " nach ", localPath + " herunter")
  with urllib.request.urlopen(URL) as url:
    with open(localPath, 'wb') as f:
      f.write(url.read())

In [0]:
def getFileNameFromPath(path):
  filename = path.split("/")[len(path.split("/")) - 1]
  return filename

In [0]:
import os

def createLocalDirectory(dir):
  
  print("Erstelle lokales Verzeichnis:", dir, end='\t')
  
  try:  
    os.makedirs(dir)

  except OSError:  
    print (" - Failed")
  else:
    print(" - OK")

In [0]:
import os

def findFilesInPath(dir, extensions):

  files = []
  
  # r=root, d=directories, f = files
  for r, d, f in os.walk(dir):
    for file in f:
      for extension in extensions:
        if file.lower().endswith(extension.lower()):
            files.append(os.path.join(r, file))
            break
            
  return files

In [0]:
import email
from email import policy
from email.parser import BytesParser

def examineEmail(pathToEmail):
  get_list = ['from','to','subject']
  info_dict = {}
  
  with open(pathToEmail, 'rb') as fp:  # select a specific email file from the list
    msg = BytesParser(policy=policy.default).parse(fp)
    text = msg.get_body(preferencelist=('plain')).get_content()
    
    for get_item in get_list:
      info_dict[get_item] = msg[get_item]
      
    info_dict["body"] = text
    
    return info_dict

In [0]:
localEmailPath = "/tmp/email_segmentation"
createLocalDirectory(localEmailPath)

In [0]:
from urllib.request import urlopen

filesDict = {}

data = urlopen("https://raw.githubusercontent.com/ml2-picme/PicMe/master/input/emails.txt")
for line in data:
  if not line.startswith(b'/'):  # Ignore Lines that begin with a comment (#)
    url = line.decode("utf-8").split("\n")[0]  # Normalization
    
    filename = getFileNameFromPath(url)

    localPath = localEmailPath + "/" + filename

    downloadEmailFromUrl(url, localPath)
    
    filesDict[localPath] = url
    
for x, y in filesDict.items():
  print(x, "->", y)




In [0]:
extensionsToCheck = [".eml"]
emailList = findFilesInPath(localEmailPath, extensionsToCheck)

emailsDict = {}

for email in emailList:
  emailsDict[email] = examineEmail(email)
  
for fileName, emailDict in emailsDict.items():
  for item, value in emailDict.items():
    print(item, "->", value)
  print("============")
  